In [ ]:
# Install the Kaggle library
!pip install kaggle

# Upload the Kaggle API key
from google.colab import files
files.upload()  # Select the kaggle.json file downloaded from the Kaggle account settings page

In [2]:
!mkdir -p ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sachinkumar413/alzheimer-mri-dataset

In [ ]:
!unzip alzheimer-mri-dataset.zip

In [ ]:
!pip install torch

In [ ]:
!pip install transformers

In [9]:
# Import necessary libraries
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, roc_curve
import matplotlib.pyplot as plt
from transformers import AutoModelForImageClassification as BeitForImageClassification
from collections import Counter
from sklearn.model_selection import train_test_split

In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [11]:
!ls

Alzheimer_200epoch_91.5625.pt  alzheimer-mri-dataset.zip  Dataset  kaggle.json	sample_data


In [12]:
# Load the dataset
full_data = torchvision.datasets.ImageFolder(root='Dataset', transform=transform)

In [14]:
full_data

Dataset ImageFolder
    Number of datapoints: 6400
    Root location: Dataset
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [5]:
len(full_data)

6400

In [15]:
# Split the dataset into train, validation, and test sets
train_data, temp_data = train_test_split(full_data, test_size=0.3)
val_data, test_data = train_test_split(temp_data, test_size=0.5)

In [17]:
len(test_data)

960

In [22]:
# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [29]:
len(test_loader)

30

In [21]:
full_data.imgs[6396:6399]

[('Dataset/Very_Mild_Demented/verymild_996.jpg', 3),
 ('Dataset/Very_Mild_Demented/verymild_997.jpg', 3),
 ('Dataset/Very_Mild_Demented/verymild_998.jpg', 3)]

In [24]:
# Calculate class weights
class_counts = Counter(img_tuple[1] for img_tuple in full_data.imgs)
class_weights = {cls: len(full_data) / count for cls, count in class_counts.items()}
class_weights = [class_weights[i] for i in range(len(class_weights))]

In [25]:
# Convert to tensor
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [ ]:
# Move to GPU if available
# if torch.cuda.is_available():
class_weights = class_weights.to('cuda')

In [11]:
from torch import nn

# Load the model
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224')

# Redefine the final layer to match the number of classes in your dataset
num_classes = len(class_weights)  # Number of classes in your dataset
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [12]:
# Define loss function with class weights and optimizer
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Move the model to the GPU
model = model.to('cuda')

In [14]:
num_epochs = 200

In [15]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to('cuda')
        labels = labels.to('cuda')
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epochs Done Till Now:', epoch+1)

Epochs Done Till Now: 1
Epochs Done Till Now: 2
Epochs Done Till Now: 3
Epochs Done Till Now: 4
Epochs Done Till Now: 5
Epochs Done Till Now: 6
Epochs Done Till Now: 7
Epochs Done Till Now: 8
Epochs Done Till Now: 9
Epochs Done Till Now: 10
Epochs Done Till Now: 11
Epochs Done Till Now: 12
Epochs Done Till Now: 13
Epochs Done Till Now: 14
Epochs Done Till Now: 15
Epochs Done Till Now: 16
Epochs Done Till Now: 17
Epochs Done Till Now: 18
Epochs Done Till Now: 19
Epochs Done Till Now: 20
Epochs Done Till Now: 21
Epochs Done Till Now: 22
Epochs Done Till Now: 23
Epochs Done Till Now: 24
Epochs Done Till Now: 25
Epochs Done Till Now: 26
Epochs Done Till Now: 27
Epochs Done Till Now: 28
Epochs Done Till Now: 29
Epochs Done Till Now: 30
Epochs Done Till Now: 31
Epochs Done Till Now: 32
Epochs Done Till Now: 33
Epochs Done Till Now: 34
Epochs Done Till Now: 35
Epochs Done Till Now: 36
Epochs Done Till Now: 37
Epochs Done Till Now: 38
Epochs Done Till Now: 39
Epochs Done Till Now: 40
Epochs Do

In [16]:
# Validation loop
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Validation Accuracy: {} %'.format(100 * correct / total))

Validation Accuracy: 91.5625 %


In [18]:
# Testing loop and metrics
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to('cuda')
        labels = labels.to('cuda')
        outputs = model(images)
        _, predicted = torch.max(outputs.logits.data, 1)
        all_labels.extend(labels)
        all_predictions.extend(predicted)

In [19]:
# Move tensors to CPU memory
all_labels = [label.cpu() for label in all_labels]
all_predictions = [prediction.cpu() for prediction in all_predictions]

# Confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[110   0   5   8]
 [  0  10   0   1]
 [  2   0 462  16]
 [  7   0  17 322]]


In [26]:
len(all_predictions)

960

In [24]:
from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(all_labels, all_predictions))


              precision    recall  f1-score   support

           0       0.92      0.89      0.91       123
           1       1.00      0.91      0.95        11
           2       0.95      0.96      0.96       480
           3       0.93      0.93      0.93       346

    accuracy                           0.94       960
   macro avg       0.95      0.92      0.94       960
weighted avg       0.94      0.94      0.94       960



In [ ]:
torch.save(model.state_dict(), 'alzheimer_model.pth')